In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
# from transformers import BertTokenizer, TFBertModel
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch.nn as nn



In [2]:
train=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/train_pp2.csv' ,delimiter=',')
test=pd.read_csv('C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/461p/ECE-461P-Term-Project/test_pp2.csv' ,delimiter=',')



In [3]:
display(train.head())

,train_id,item_condition_id,price,shipping,vec0,vec1,vec2,vec3,vec4,vec5,...,cat_Electronics,cat_Fashion,cat_Food and Beverage,cat_Furniture and Decor,cat_Health and Wellness,cat_Household Items,cat_Media,cat_Office Equipment,cat_Pet Care,cat_Toys and Hobbies
0,0,3,10.0,1,-0.007633,0.013877,0.001542,0.002168,-0.002364,-0.014765,...,0,1,0,0,0,0,0,0,0,0
1,1,3,52.0,0,0.013427,-0.005142,0.006128,0.001119,-0.010722,0.013957,...,1,0,0,0,0,0,0,0,0,0
2,2,1,10.0,1,0.001426,-0.006458,0.002439,0.008610,0.012070,-0.011089,...,0,1,0,0,0,0,0,0,0,0
3,3,1,35.0,1,0.005893,-0.008465,0.012312,-0.003756,0.010533,-0.000997,...,0,0,0,1,0,0,0,0,0,0
4,4,1,44.0,0,0.008761,0.012107,-0.013877,-0.008480,-0.004083,-0.001077,...,0,1,0,0,0,0,0,0,0,0


In [5]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
# Separate features and target
X = train.drop(columns=['price'])
y = (train['price']) # talk about this 


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
display(y_train)

1142652    18.0
1142448    14.0
220982     15.0
479456     23.0
1299063    36.0
           ... 
259178     44.0
1414414    26.0
131932     51.0
671155     10.0
121958     10.0
Name: price, Length: 1185328, dtype: float64

create data for pytorch

In [7]:
# # Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [8]:
X_train_tensor = torch.tensor(X_train.astype(np.float32)).to(device)
X_test_tensor = torch.tensor(X_test.astype(np.float32)).to(device)
y_train_tensor = torch.tensor(y_train.values.astype(np.float32)).to(device)
y_test_tensor = torch.tensor(y_test.values.astype(np.float32)).to(device)

# Reshape y tensors to have the correct shape (n_samples, 1)
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)


In [9]:

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

Model Building

In [10]:
class CNNModel(nn.Module):
    def __init__(self, num_features):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(16 * num_features, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  
        x = self.conv1(x)
        x = self.relu(x)
        x = self.flatten(x)
        x = self.fc1(x)
        return torch.exp(x)

class RegularizedModel(nn.Module):
    def __init__(self, num_features):
        super(RegularizedModel, self).__init__()
        self.fc1 = nn.Linear(num_features, 128)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return  torch.exp(x)

class ComplexModel(nn.Module):
    def __init__(self, num_features):
        super(ComplexModel, self).__init__()
        self.fc1 = nn.Linear(num_features, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return  torch.exp(x)
class EnhancedModel(nn.Module):
    def __init__(self, num_features):
        super(EnhancedModel, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(num_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5)
        ).to(device=device)
        self.layer2 = nn.Sequential(
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.4)
        ).to(device=device)
        self.layer3 = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3)
        ).to(device=device)
        self.final_layer = nn.Linear(128, 1).to(device=device)

    def forward(self, x):
        x = self.layer1(x).to(device=device)
        x = self.layer2(x).to(device=device)
        x = self.layer3(x).to(device=device)
        x = self.final_layer(x).to(device=device)
        return  torch.exp(x)



In [11]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

def rmsle_loss(y_pred, y_true):
    # Ensure the inputs are float tensors (required for MSELoss and logarithmic operations)
    y_true = y_true.float()
    y_pred = y_pred.float()

    # Adjust predictions to ensure they are all positive (since log(0) is undefined)
    y_pred = torch.clamp(y_pred, min=1e-6)  # Adding a small epsilon to avoid log(0)
    
    # Compute the RMSLE using the logarithmic differences
    return torch.sqrt(torch.mean((torch.log1p(y_pred) - torch.log1p(y_true)) ** 2))

def train_model(model, train_loader, test_loader, num_epochs=5):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True, min_lr=1e-6)

    # Initialize minimum loss to a large value
    min_train_rmsle = float('inf')
    min_test_rmsle = float('inf')

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = rmsle_loss(outputs, targets)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_train_rmsle = total_train_loss / len(train_loader)

        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                test_loss = rmsle_loss(outputs, targets)
                total_test_loss += test_loss.item()

        avg_test_rmsle = total_test_loss / len(test_loader)
        
        # Update the learning rate scheduler
        scheduler.step(avg_test_rmsle)

        if avg_train_rmsle < min_train_rmsle:
            min_train_rmsle = avg_train_rmsle
        if avg_test_rmsle < min_test_rmsle:
            min_test_rmsle = avg_test_rmsle

        print(f'Epoch {epoch+1}: Train RMSLE = {avg_train_rmsle}, Test RMSLE = {avg_test_rmsle}')
    
    print(f'Minimum Train RMSLE so far: {min_train_rmsle}')
    print(f'Minimum Test RMSLE so far: {min_test_rmsle}')

In [14]:
num_features = X_train.shape[1]


In [130]:

# Example usage
model = CNNModel(num_features).to(device)
train_model(model, train_loader, test_loader)
# Minimum Train RMSLE so far: 0.7004365426701981
# Minimum Test RMSLE so far: 0.698955543707679

Epoch 1: Train RMSLE = 5137485853381.73, Test RMSLE = 0.784662239039544
Epoch 2: Train RMSLE = 0.7680806976261667, Test RMSLE = 0.747738756775161
Epoch 3: Train RMSLE = 0.7413838792037748, Test RMSLE = 0.7395818479643916
Epoch 4: Train RMSLE = 0.7289735549106163, Test RMSLE = 0.7160217967282565
Epoch 5: Train RMSLE = 2.8293548017091776, Test RMSLE = 0.7076882655317854
Minimum Train RMSLE so far: 0.7289735549106163
Minimum Test RMSLE so far: 0.7076882655317854


In [ ]:
model2 = RegularizedModel(num_features).to(device)
train_model(model2, train_loader, test_loader)
# doesnt work


In [131]:
model3 = ComplexModel(num_features).to(device)
train_model(model3, train_loader, test_loader)
# Minimum Train RMSLE so far: 0.688113078240873
# Minimum Test RMSLE so far: 0.687606498730978

# Minimum Train RMSLE so far: 0.688113078240873
# Minimum Test RMSLE so far: 0.687606498730978

Epoch 1: Train RMSLE = 0.7043774527888352, Test RMSLE = 0.6922612426103874
Epoch 2: Train RMSLE = 0.6906035386291811, Test RMSLE = 0.6879250371631189
Epoch 3: Train RMSLE = 0.6895467307428441, Test RMSLE = 0.6880920242218136
Epoch 4: Train RMSLE = 0.6890383984353101, Test RMSLE = 0.688188678230845
Epoch 5: Train RMSLE = 0.6887633485748627, Test RMSLE = 0.6880506542608821
Minimum Train RMSLE so far: 0.6887633485748627
Minimum Test RMSLE so far: 0.6879250371631189


In [111]:
model4 = EnhancedModel(num_features).to(device)
train_model(model4, train_loader, test_loader)
# Minimum Train RMSLE so far: 0.69227513503413
# Minimum Test RMSLE so far: 0.6879885306087536


Epoch 1: Train RMSLE = 0.7159452734736425, Test RMSLE = 0.688680603794546
Epoch 2: Train RMSLE = 0.6935721546448523, Test RMSLE = 0.6879314284774094
Epoch 3: Train RMSLE = 0.6920516048398906, Test RMSLE = 0.6879945102828905
Epoch 4: Train RMSLE = 0.6916209922841067, Test RMSLE = 0.6877204380483819
Epoch 5: Train RMSLE = 0.6913184327930799, Test RMSLE = 0.6879418392375488
Epoch 6: Train RMSLE = 0.6912575840505898, Test RMSLE = 0.6878238535743376
Epoch 7: Train RMSLE = 0.6909255641713963, Test RMSLE = 0.6880650057748388
Epoch 8: Train RMSLE = 0.690915531024299, Test RMSLE = 0.6879392043673125
Epoch 9: Train RMSLE = 0.6908022080224827, Test RMSLE = 0.6896664626370288
Epoch 10: Train RMSLE = 0.6908659141915617, Test RMSLE = 0.6877503546054974
Epoch 11: Train RMSLE = 0.6907172332162843, Test RMSLE = 0.6877906458330474
Epoch 12: Train RMSLE = 0.6905958335614347, Test RMSLE = 0.6876177308601119
Epoch 13: Train RMSLE = 0.6905114932352714, Test RMSLE = 0.6875397739825355
Epoch 14: Train RMSLE =

Finished Base neural network

In [93]:
class ResidualBlock(nn.Module):
    def __init__(self, num_features):
        super(ResidualBlock, self).__init__()
        self.fc1 = nn.Linear(num_features, num_features)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(num_features, num_features)
        self.bn = nn.BatchNorm1d(num_features)

    def forward(self, x):
        residual = x
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.bn(out)
        out += residual
        out = self.relu(out)
        return  out

class ResNet(nn.Module):
    def __init__(self, num_features):
        super(ResNet, self).__init__()
        self.block1 = ResidualBlock(num_features)
        self.block2 = ResidualBlock(num_features)
        self.final_layer = nn.Linear(num_features, 1)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.final_layer(x)
        return  x


In [94]:
model5 = ResidualBlock(num_features).to(device)
train_model(model5, train_loader, test_loader)
#Epoch 2, RMSLE: 0.7510158108007159



Epoch 1: Train RMSLE = 1.426390644467216, Test RMSLE = 1.0603972458087763
Epoch 2: Train RMSLE = 0.7680296310273471, Test RMSLE = 0.6909068401020461
Epoch 3: Train RMSLE = 0.6924842647304223, Test RMSLE = 0.6904345765095297
Epoch 4: Train RMSLE = 0.6919504568422467, Test RMSLE = 0.690175749543315
Epoch 5: Train RMSLE = 0.6916262712503611, Test RMSLE = 0.6896854765380183
Epoch 6: Train RMSLE = 0.6913382063403216, Test RMSLE = 0.6895079688798323
Epoch 7: Train RMSLE = 0.6912930727336482, Test RMSLE = 0.6895803656067866
Epoch 8: Train RMSLE = 0.6911249879475488, Test RMSLE = 0.6890395522774025
Epoch 9: Train RMSLE = 0.6908792133906331, Test RMSLE = 0.6891707153550443
Epoch 10: Train RMSLE = 0.6908167007674757, Test RMSLE = 0.689398987861875
Minimum Train RMSLE so far: 0.6908167007674757
Minimum Test RMSLE so far: 0.6890395522774025


In [ ]:
model6 = ResNet(num_features).to(device)
train_model(model6, train_loader, test_loader)
#Epoch 2, RMSLE: 0.7510158108007159

In [96]:


class MLP(nn.Module):
    def __init__(self, num_features):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(num_features, 512)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 256)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(256, 128)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout3(x)
        x = self.fc4(x)
        return torch.exp(x)


In [132]:
model7 = MLP(num_features).to(device)
train_model(model7, train_loader, test_loader)

# Minimum Train RMSLE so far: 0.6939673007973121
# Minimum Test RMSLE so far: 0.6892061752535257

Epoch 1: Train RMSLE = 0.7789893851209541, Test RMSLE = 0.6917615989659522
Epoch 2: Train RMSLE = 0.6964964387445288, Test RMSLE = 0.6951838828425406
Epoch 3: Train RMSLE = 19.46874885803892, Test RMSLE = 0.6905204756127091
Epoch 4: Train RMSLE = 0.6960191253654566, Test RMSLE = 0.6944242547099468
Epoch 5: Train RMSLE = 0.6958181958599924, Test RMSLE = 0.6898327518606052
Minimum Train RMSLE so far: 0.6958181958599924
Minimum Test RMSLE so far: 0.6898327518606052


In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AdvancedMLP(nn.Module):
    def __init__(self, num_features):
        super(AdvancedMLP, self).__init__()
        self.fc1 = nn.Linear(num_features, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.dropout1 = nn.Dropout(0.1)
        
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.dropout2 = nn.Dropout(0.05)
        
        self.fc3 = nn.Linear(512, 256)
        self.bn3 = nn.BatchNorm1d(256)
        self.dropout3 = nn.Dropout(0.01)
        
        self.fc4 = nn.Linear(256, 128)
        self.bn4 = nn.BatchNorm1d(128)
        
        self.final_layer = nn.Linear(128, 1)

    def forward(self, x):
        x = F.leaky_relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.leaky_relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = F.leaky_relu(self.bn3(self.fc3(x)))
        x = self.dropout3(x)
        x = F.leaky_relu(self.bn4(self.fc4(x)))
        x = self.final_layer(x)
        return x


In [23]:
model = AdvancedMLP(num_features).to(device)
train_model(model, train_loader, test_loader, num_epochs=1000).to(device)
# Minimum Train RMSLE so far: 0.6882604910622443
# Minimum Test RMSLE so far: 0.687623768491432

Epoch 1: Train RMSLE = 0.7023633642588892, Test RMSLE = 0.6903367664844371
Epoch 2: Train RMSLE = 0.6899257162323872, Test RMSLE = 0.6880816763612422
Epoch 3: Train RMSLE = 0.6892484756687741, Test RMSLE = 0.6876663033235307
Epoch 4: Train RMSLE = 0.6888325244814253, Test RMSLE = 0.6893707580866039
Epoch 5: Train RMSLE = 0.6885519486241887, Test RMSLE = 0.6881060414490857
Epoch 6: Train RMSLE = 0.6883642813243617, Test RMSLE = 0.6883526197122822
Epoch 7: Train RMSLE = 0.6875517218975171, Test RMSLE = 0.6877412989790305
Epoch 8: Train RMSLE = 0.6873011699644409, Test RMSLE = 0.6875942824881012
Epoch 9: Train RMSLE = 0.6870216123711875, Test RMSLE = 0.6877278189400673
Epoch 10: Train RMSLE = 0.6868236210261776, Test RMSLE = 0.687866064763898
Epoch 11: Train RMSLE = 0.6865404002864458, Test RMSLE = 0.688016016831848
Epoch 12: Train RMSLE = 0.6858599585614292, Test RMSLE = 0.6880781474921235
Epoch 13: Train RMSLE = 0.6855883740862582, Test RMSLE = 0.6882511685461391
Epoch 14: Train RMSLE =

KeyboardInterrupt: 

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomMLP(nn.Module):
    def __init__(self, num_features):
        super(CustomMLP, self).__init__()
        self.fc1 = nn.Linear(num_features, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 32)
        self.dropout = nn.Dropout(0.2)
        
        self.final_layer = nn.Linear(32, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.relu(self.fc6(x))
        
        x = self.dropout(x)
        x = self.final_layer(x)
        return x


In [ ]:
model10 = CustomMLP(num_features).to(device)
train_model(model10, train_loader, test_loader, num_epochs=100).to(device)

# Minimum Train RMSLE so far: 0.6903926233359389
# Minimum Test RMSLE so far: 0.6875400086750199

xgboost

In [24]:
!pip install xgboost

  Using cached xgboost-2.0.3-py3-none-win_amd64.whl.metadata (2.0 kB)
Using cached xgboost-2.0.3-py3-none-win_amd64.whl (99.8 MB)


In [7]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import make_scorer

# Assume `train` is your DataFrame loaded with the data
X = train.drop(columns=['price'])
y = np.log1p(train['price'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [8]:
def rmsle(preds, dtrain):
    labels = dtrain.get_label()
    # prevent negative predictions
    preds = np.clip(preds, a_min=0, a_max=None)
    return 'RMSLE', np.sqrt(np.mean(np.power((preds-labels), 2)))


In [9]:
from scipy.stats import randint, uniform

param_dist = {
    'max_depth': randint(0, 100),
    'min_child_weight': randint(0, 100),
    'subsample': uniform(0.6, 0.6),
    'colsample_bytree': uniform(0.6, 0.6),
    'learning_rate': uniform(0.01, 0.8),
    'n_estimators': randint(0,1000)
}

# Create the RMSLE scorer
def rmsle_scorer(y_true, y_pred):
    return np.sqrt(np.mean(np.power(y_pred - y_true, 2)))

rmsle_scoring = make_scorer(rmsle_scorer, greater_is_better=False)

# Initialize the XGBoost Regressor
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_jobs=10)

# Setup RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_dist, 
                                   n_iter=100, scoring=rmsle_scoring, cv=3, verbose=3, random_state=42,n_jobs=10)

# Fit the model
random_search.fit(X_train, y_train)


Fitting 3 folds for each of 100 candidates, totalling 300 fits


KeyboardInterrupt: 

In [ ]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
test_rmsle = rmsle_scorer(y_test, y_pred)

print("Best model parameters:", random_search.best_params_)
print("Test RMSLE:", test_rmsle)


Best model parameters: {'colsample_bytree': 0.8253152871382157, 'learning_rate': 0.14562495076197482, 'max_depth': 4, 'min_child_weight': 5, 'n_estimators': 200, 'subsample': 0.8736932106048627}
Test RMSLE: 0.7570922261734949
